# 🔋 Battery RAG System - Following Architecture Diagram

## 🏗️ RAG Architecture Flow:
1. **Data Input** → **Document Chunking** → **Embedding Model** → **Vector Store**
2. **User Query** → **Semantic Search** → **Retrieved Context**  
3. **Query + Context** → **Prompt Template** → **LLM** → **Response**

Each step implemented in small, clear cells for easy understanding and API integration.

## 📦 Step 1: Basic Imports

In [43]:
# Basic data processing imports
import pandas as pd
import numpy as np
from datetime import datetime
print("✅ Basic imports loaded")

✅ Basic imports loaded


## 🧠 Step 2: Vector & LLM Imports

In [44]:
# Vector database and embeddings
import chromadb
from sentence_transformers import SentenceTransformer
print("✅ Vector processing imports loaded")

✅ Vector processing imports loaded


In [45]:
# LangChain for RAG pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
print("✅ LangChain imports loaded")

✅ LangChain imports loaded


## 📊 Step 3: Data Input & Loading

In [46]:
# Load battery dataset
print("📊 Loading battery data...")
df = pd.read_csv('battery_data_10000_rows.csv')
df = df.apply(pd.to_numeric, errors='ignore').dropna().reset_index(drop=True)
print(f"✅ Loaded {len(df)} battery records")
print(f"📋 Columns: {list(df.columns)}")

📊 Loading battery data...
✅ Loaded 10000 battery records
📋 Columns: ['Cell_ID', 'Cell_Type', 'Nominal_Voltage_V', 'Capacity_Ah', 'Internal_Resistance_mOhm', 'Gravimetric_Energy_Density_Wh/kg', 'Volumetric_Energy_Density_Wh/L', 'Thermal_Runaway_Temp_C', 'Cathode_Material', 'Anode_Material', 'Separator_Material']


C:\Users\gaura\AppData\Local\Temp\ipykernel_17960\3792319427.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore').dropna().reset_index(drop=True)


## 📄 Step 4: Document Chunking & Formatting

In [47]:
# Document formatting function
def format_battery_document(row):
    """Convert battery row into structured text document"""
    return f"""Battery ID: {row.iloc[0]}
Type: {row.iloc[1] if len(row) > 1 else 'Unknown'}
Voltage: {row.iloc[2] if len(row) > 2 else 'Unknown'} V
Capacity: {row.iloc[3] if len(row) > 3 else 'Unknown'} Ah
Energy Density: {row.iloc[4] if len(row) > 4 else 'Unknown'} Wh/kg"""

print("📄 Document formatting function ready")

📄 Document formatting function ready


In [48]:
# Create structured documents from data
print("📄 Creating structured documents...")
documents = [format_battery_document(row) for _, row in df.iterrows()]
print(f"✅ Created {len(documents)} structured documents")
print(f"📋 Sample document:\n{documents[0]}")

📄 Creating structured documents...
✅ Created 10000 structured documents
📋 Sample document:
Battery ID: BAT-00001
Type: Pouch
Voltage: 3.71 V
Capacity: 4.14 Ah
Energy Density: 17.4 Wh/kg
✅ Created 10000 structured documents
📋 Sample document:
Battery ID: BAT-00001
Type: Pouch
Voltage: 3.71 V
Capacity: 4.14 Ah
Energy Density: 17.4 Wh/kg


## 🧠 Step 5: Embedding Model

In [49]:
# Initialize embedding model
print("🧠 Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded")

🧠 Loading embedding model...
✅ Embedding model loaded
✅ Embedding model loaded


In [50]:
# Create embeddings for all documents
print("🧠 Creating embeddings...")
embeddings = embedding_model.encode(documents, show_progress_bar=True)
print(f"✅ Created embeddings with shape: {embeddings.shape}")

🧠 Creating embeddings...


Batches: 100%|██████████| 313/313 [00:45<00:00,  6.81it/s]

✅ Created embeddings with shape: (10000, 384)


## 🗄️ Step 6: Vector Store Setup

In [51]:
# Initialize ChromaDB vector store
print("🗄️ Setting up vector store...")
chroma_client = chromadb.Client()

# Clean up existing collection
try:
    chroma_client.delete_collection(name="battery_rag")
except:
    pass

collection = chroma_client.create_collection(name="battery_rag")
print("✅ Vector store initialized")

🗄️ Setting up vector store...
✅ Vector store initialized


In [52]:
# Store documents and embeddings in batches
print("💾 Storing documents in vector store...")
batch_size = 1000

for i in range(0, len(documents), batch_size):
    end_idx = min(i + batch_size, len(documents))
    collection.add(
        documents=documents[i:end_idx],
        embeddings=embeddings[i:end_idx].tolist(),
        ids=[str(j) for j in range(i, end_idx)]
    )
    print(f"   Batch {i//batch_size + 1}: {end_idx - i} documents")

print(f"✅ Stored {collection.count()} documents in vector store")

💾 Storing documents in vector store...
   Batch 1: 1000 documents
   Batch 2: 1000 documents
   Batch 3: 1000 documents
   Batch 4: 1000 documents
   Batch 5: 1000 documents
   Batch 6: 1000 documents
   Batch 7: 1000 documents
   Batch 8: 1000 documents
   Batch 9: 1000 documents
   Batch 10: 1000 documents
✅ Stored 10000 documents in vector store


## 🔍 Step 7: Semantic Search Function

In [53]:
# Semantic search function
def semantic_search(query, top_k=3):
    """Perform semantic search to retrieve relevant documents"""
    print(f"🔍 Searching for: '{query}'")
    
    # Create query embedding
    query_embedding = embedding_model.encode([query])
    
    # Search vector store
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=top_k
    )
    
    retrieved_docs = results['documents'][0]
    print(f"✅ Retrieved {len(retrieved_docs)} relevant documents")
    return retrieved_docs

print("🔍 Semantic search function ready")

🔍 Semantic search function ready


## 📝 Step 8: Prompt Template

In [54]:
# Create prompt template for LLM
prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template="""You are a battery engineering expert. Use the provided context to answer the user's question.

Context (Retrieved Battery Data):
{context}

User Question: {query}

Instructions:
- Answer based only on the provided context
- Be technical and precise
- If calculating configurations, show step-by-step work
- For 2S3P: 2 in series (voltage adds), 3 in parallel (capacity adds)

Answer:"""
)

print("📝 Prompt template created")

📝 Prompt template created


## 🤖 Step 9: LLM Setup (API Ready)

In [65]:
# LLM Setup - Google Gemini Integration
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set your API key (replace with your actual key)
google_api_key = "gemini_api_key"

try:
    # Use a valid model name - gemini-1.5-flash is currently available
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    
    # Test the connection
    test_response = llm.invoke("Hello, are you working?")
    print("✅ Google Gemini LLM initialized successfully!")
    print(f"🧪 Test response: {test_response.content[:50]}...")
    
except Exception as e:
    print(f"❌ Error with gemini-1.5-flash: {e}")
    print("🔄 Trying alternative model...")
    
    print("❌ API key is not working. Please check your API key and try again.")
    llm = None

print("🤖 LLM ready for RAG pipeline")

✅ Google Gemini LLM initialized successfully!
🧪 Test response: Yes, I am working.  I'm ready to assist you with y...
🤖 LLM ready for RAG pipeline


## ⛓️ Step 10: RAG Pipeline

In [61]:
# Complete RAG pipeline function
def rag_query(user_query):
    """Complete RAG pipeline: Query → Search → Context → Prompt → LLM → Response"""
    
    print(f"\n🔄 RAG Pipeline Processing: '{user_query}'")
    print("=" * 50)
    
    # Step 1: Semantic Search
    print("🔍 Step 1: Semantic Search")
    retrieved_docs = semantic_search(user_query, top_k=3)
    
    # Step 2: Format Context
    print("📄 Step 2: Formatting Context")
    context = "\n---\n".join(retrieved_docs)
    
    # Step 3: Create Prompt
    print("📝 Step 3: Creating Prompt")
    formatted_prompt = prompt_template.format(query=user_query, context=context)
    
    # Step 4: LLM Processing
    print("🤖 Step 4: LLM Processing")
    try:
        llm_response = llm.invoke(formatted_prompt)
        
        # Handle different response types
        if hasattr(llm_response, 'content'):
            response = llm_response.content  # Gemini response
        else:
            response = str(llm_response)  # Fallback
            
    except Exception as e:
        print(f"⚠️ LLM Error: {e}")
        response = "Error occurred while processing with LLM. Please check your API key and model availability."
    
    print("✅ RAG Pipeline Complete\n")
    
    return {
        "query": user_query,
        "retrieved_docs": retrieved_docs,
        "response": response,
        "context": context
    }

print("⛓️ RAG pipeline function ready")

⛓️ RAG pipeline function ready


## 🧪 Step 11: Test the RAG System

In [64]:
# Test the complete RAG system
test_query = "Calculate 2S3P battery configuration"

result = rag_query(test_query)

print("📋 RESULT:")
print(f"Query: {result['query']}")
print(f"Response: {result['response']}")
print(f"\n📊 Retrieved {len(result['retrieved_docs'])} documents for context")


🔄 RAG Pipeline Processing: 'Calculate 2S3P battery configuration'
🔍 Step 1: Semantic Search
🔍 Searching for: 'Calculate 2S3P battery configuration'
✅ Retrieved 3 relevant documents
📄 Step 2: Formatting Context
📝 Step 3: Creating Prompt
🤖 Step 4: LLM Processing
✅ RAG Pipeline Complete

📋 RESULT:
Query: Calculate 2S3P battery configuration
Response: To calculate a 2S3P configuration, we need to select three batteries from the provided data to create a parallel group, and then combine two of these parallel groups in series.  Since we only have data for three unique batteries, we'll use all three to create a 2S3P configuration which is effectively a 2S1P + 1S1P series connection. This is not ideal, as a proper 2S3P would require 6 identical batteries. However, we will proceed to show the calculation based on the available data.


**Step 1: Select Batteries for Parallel Groups**

We will arbitrarily select BAT-02123, BAT-02705, and BAT-04335.  We'll form two groups. Group 1 will be BAT-021

## 🎯 Direct Query Cell - Write Your Question Here

In [72]:
# ✏️ WRITE YOUR QUERY HERE ✏️
# Change the query below to ask any battery-related question
my_query = "Compare lithium-ion vs lithium-polymer batteries"

# Execute RAG pipeline
print(f"🎯 Processing your query: '{my_query}'")
print("=" * 60)

result = rag_query(my_query)

print("\n" + "="*60)
print("📋 COMPLETE RESULT:")
print("="*60)
print(f"❓ Your Query: {result['query']}")
print(f"\n🤖 AI Response:\n{result['response']}")
print(f"\n📊 Retrieved {len(result['retrieved_docs'])} relevant documents")
print("\n📄 Context Used:")
print("-" * 40)
for i, doc in enumerate(result['retrieved_docs'][:2], 1):
    print(f"Document {i}:\n{doc[:200]}...\n")
print("="*60)

🎯 Processing your query: 'Compare lithium-ion vs lithium-polymer batteries'

🔄 RAG Pipeline Processing: 'Compare lithium-ion vs lithium-polymer batteries'
🔍 Step 1: Semantic Search
🔍 Searching for: 'Compare lithium-ion vs lithium-polymer batteries'
✅ Retrieved 3 relevant documents
📄 Step 2: Formatting Context
📝 Step 3: Creating Prompt
🤖 Step 4: LLM Processing
✅ RAG Pipeline Complete


📋 COMPLETE RESULT:
❓ Your Query: Compare lithium-ion vs lithium-polymer batteries

🤖 AI Response:
The provided data only includes information on cylindrical lithium-ion batteries.  There is no data on lithium-polymer batteries, therefore a comparison cannot be made.  The context only allows for analysis of the three listed cylindrical lithium-ion batteries (BAT-00185, BAT-00190, and BAT-00118), showing variations in capacity and energy density.

📊 Retrieved 3 relevant documents

📄 Context Used:
----------------------------------------
Document 1:
Battery ID: BAT-00185
Type: Cylindrical
Voltage: 3.61 V
Cap

## 🎯 Step 12: Interactive Chat Function

In [69]:
# Interactive chat function
def battery_chat():
    """Interactive chat using the RAG system"""
    print("🔋 Battery RAG Chat - Type 'quit' to exit\n")
    
    while True:
        user_input = input("❓ Your question: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break
            
        if not user_input:
            continue
            
        # Process through RAG pipeline
        result = rag_query(user_input)
        print(f"\n🤖 Response: {result['response']}\n")

print("💬 Interactive chat function ready")
print("🚀 Run: battery_chat() to start interactive mode")

💬 Interactive chat function ready
🚀 Run: battery_chat() to start interactive mode
